## Урок 6. Классификация текста. Анализ тональности текста

Взять ноутбук colab\_text\_classification\_part1.ipynb который разбирали на занятии (лежит в архиве lection6\_materials_actual.zip) и добавить пункты которые мы пропустили

1.  Проверьте повысилось ли качество на стандартных подходах при лемматизации/и без неё
2.  Удалите/(замените на тег) из текстов сущности(имена, локации, что-то ещё). Запустите классификатор и модельки на сеточках
3.  Сделайте выводы

Полезные ссылки:
https://paperswithcode.com/

In [1]:
# !pip install spacy

# !python -m spacy download en_core_web_lg

In [1]:
import pickle
import re
import spacy
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm

from utils import apostrophe_dict, emoticon_dict, short_word_dict  # см. файл utils.py
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

tqdm.pandas()

# Классификация текстов

Начнём с самого простого - анализа тональности текста.

Будем классифицировать отзывы с IMDB на положительные/отрицательные.

Датасет взят с http://ai.stanford.edu/~amaas/data/sentiment/

In [2]:
import pandas as pd

train_df = pd.read_csv("train.tsv", delimiter="\t")
test_df = pd.read_csv("test.tsv", delimiter="\t")

print('Train size = {}'.format(len(train_df)))
print('Test size = {}'.format(len(test_df)))

Train size = 25000
Test size = 25000


In [4]:
train_df.head(3)

,is_positive,review
0,0,"Dreamgirls, despite its fistful of Tony wins i..."
1,0,This show comes up with interesting locations ...
2,1,I simply love this movie. I also love the Ramo...


Посмотрите глазами на тексты? Какие есть зацепки, как определить, что это за сентимент?

Самое простое, как всегда - найти ключевые слова.

In [5]:
#@title Начинаем классифицировать! { vertical-output: true, display-mode: "form" }

positive_words = 'love', 'great', 'best', 'wonderful' #@param {type:"raw"}
negative_words = 'worst', 'awful', '1/10', 'crap' #@param {type:"raw"}

positives_count = test_df.review.apply(lambda text: sum(word in text for word in positive_words))
negatives_count = test_df.review.apply(lambda text: sum(word in text for word in negative_words))
is_positive = positives_count > negatives_count
correct_count = (is_positive == test_df.is_positive).values.sum()

accuracy = correct_count / len(test_df)

print('Test accuracy = {:.2%}'.format(accuracy))
if accuracy > 0.71:
    from IPython.display import Image, display
    display(Image('https://s3.amazonaws.com/achgen360/t/rmmoZsub.png', width=500))

Test accuracy = 66.73%


In [6]:
pattern = re.compile('<br />')

# print(train_df['review'].iloc[3])
print(pattern.subn(' ', train_df['review'].iloc[3])[0])

Spoilers ahead if you want to call them that...  I would almost recommend this film just so people can truly see a 1/10. Where to begin, we'll start from the top...  THE STORY: Don't believe the premise - the movie has nothing to do with abandoned cars, and people finially understanding what the mysterious happenings are. It's a draub, basic, go to cabin movie with no intensity or "effort".  THE SCREENPLAY: I usually give credit to indie screenwriters, it's hard work when you are starting out...but this is crap. The story is flat - it leaves you emotionless the entire movie. The dialogue is extremely weak and predictable boasting lines of "Woah, you totally freaked me out" and "I was wondering if you'd uh...if you'd like to..uh, would you come to the cabin with me?". It makes me want to rip out all my hair, one strand at a time and feed it to myself.  THE CHARACTERS: HOLY CRAP!!!! Some have described the characters as flat, I want to take it one step further and say that they actually 

In [7]:
train_df['review'] = train_df['review'].apply(lambda text: pattern.subn(' ', text)[0])
test_df['review'] = test_df['review'].apply(lambda text: pattern.subn(' ', text)[0])

In [8]:
# Теперь повторим это для всех записей.
def replase_words(text,dict_): 
    output = ''
    for word in text.split(' '):
        word = word.strip()
        if word in dict_.keys(): 
            output += ' ' + dict_[word]
        else:
            output += ' ' + word
    return output

In [9]:
def clean_text(text):
    #  Удалим @user из всех твитов с помощью паттерна "@[\w]*".
    text = re.sub("@[\w]*","",text)
    # Заменим эмодзи на соответствующие им слова.
    text = replase_words(text, emoticon_dict)
     # Заменим сокращения на их полные формы
    text = replase_words(text, apostrophe_dict)
    text = replase_words(text, short_word_dict)
 
    # Заменим пунктуацию на пробелы.
    text = re.sub("[^\w\s]"," ",text)
    # Заменим спецсимволы на пробелы.
    text = re.sub("[^a-zA-Z0-9\_]"," ",text)
    # Удалим из текста слова длиной в 1 символ. За исключением "I" наверное он должен играть какуюто роль.
    # text = ' '.join([w for w in text.split() if len(w)>1 and w != "i"])
    return text

In [10]:
train_df['review'] = train_df['review'].apply(lambda x: clean_text(x))
test_df['review'] = test_df['review'].apply(lambda x: clean_text(x))

Пора переходить к машинке!

Как будем представлять текст? Проще всего - мешком слов.

Заведём большой-большой словарь - список всех слов в обучающей выборке. Тогда каждое предложение можно представить в виде вектора, в котором будет записано, сколько раз встретилось каждое из возможных слов:

![bow](https://raw.githubusercontent.com/DanAnastasyev/DeepNLP-Course/master/Week%2001/Images/BOW.png)

Простой и приятный способ сделать это - запихнуть тексты в `CountVectorizer`.

Он имеет такую сигнатуру:

```python
CountVectorizer(input='content', encoding='utf-8', decode_error='strict', strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, token_pattern=r'(?u)\b\w\w+\b', ngram_range=(1, 1), analyzer='word', max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class ‘numpy.int64'>)
```

Для начала обратим внимание на параметры `lowercase=True` и `max_df=1.0, min_df=1, max_features=None` - они про то, что по умолчанию все слова будут приводиться к нижнему регистру и в словарь попадут все слова, встречавшиеся в текстах.

При желании можно было бы убрать слишком редкие или слишком частотные слова - пока не будем этого делать.

Посмотрим на простом примере, как он будет работать:

In [11]:
train_df

,is_positive,review
0,0,Dreamgirls despite its fistful of Tony win...
1,0,This show comes up with interesting locatio...
2,1,I simply love this movie I also love the R...
3,0,Spoilers ahead if you want to call them tha...
4,1,My all time favorite movie I have seen man...
...,...,...
24995,1,I am a big fan of the movie but not for th...
24996,0,I m not going to bother with a plot synopsi...
24997,0,This movie I do not know Why they wo...
24998,1,Saw this film on DVD yesterday and was gob ...


In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

dummy_data = ['The movie was excellent',
              'the movie was awful']

dummy_matrix = vectorizer.fit_transform(dummy_data)

print(dummy_matrix.toarray())
print(vectorizer.get_feature_names())

[[0 1 1 1 1]
 [1 0 1 1 1]]
['awful', 'excellent', 'movie', 'the', 'was']


*Как именно vectorizer определяет границы слов? Обратите внимание на параметр `token_pattern=r'(?u)\b\w\w+\b'` - как он будет работать?*

Запустим его на реальных данных:

In [13]:
vectorizer = CountVectorizer()
vectorizer.fit(train_df['review'].values)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [15]:
vectorizer.transform([train_df['review'].iloc[3]])

<1x74581 sparse matrix of type '<class 'numpy.int64'>'
	with 207 stored elements in Compressed Sparse Row format>

То, что и хотели - вектор с bow (т.е. bag-of-words) представлением исходного текста.

И чем эта информация может помочь? Ну, всё тем же - какие-то слова носят положительный окрас, какие-то - отрицательный. Большинство вообще нейтральный, да.

![bow with weights](https://github.com/DanAnastasyev/DeepNLP-Course/raw/master/Week%2001/Images/BOW_weights.png)

Хочется, наверное, подобрать коэффициенты, которые будут определять уровень окраса, да? Подбирать нужно по обучающей выборке, а не как мы перед этим делали.

Например, для выборки
```
1   The movie was excellent
0   the movie was awful
```
легко подобрать коэффициенты на глазок: что-нибудь вроде `+1` для `excellent`,  `-1` для `awful` и по нулям всем остальным.

Построим линейную модель, которая станет этим заниматься. Она будет учиться строить разделяющую гиперплоскость в пространстве bow-векторов.

Проверим, как справится логистическая регрессия с нашей супер-выборкой из пары предложений

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

dummy_data = ['The movie was excellent',
              'the movie was awful']
dummy_labels = [1, 0]

vectorizer = CountVectorizer()
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(dummy_data, dummy_labels)

print(vectorizer.get_feature_names())
print(classifier.coef_)

['awful', 'excellent', 'movie', 'the', 'was']
[[-0.40105812  0.40105812  0.          0.          0.        ]]


Получилось что надо.

Запустим теперь её на реальных данных.

In [17]:
model.fit(train_df['review'], train_df['is_positive'])

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='warn', n_jobs=None,
             

In [7]:
from sklearn.metrics import accuracy_score

def eval_model(model, test_df):
    preds = model.predict(test_df['review'])
    print('Test accuracy = {:.2%}'.format(accuracy_score(test_df['is_positive'], preds)))

In [ ]:
eval_model(model, test_df)

Прогресс!

Хочется как-то посмотреть, что заинтересовало классификатор. К счастью, сделать это совсем просто:

In [19]:
import eli5
eli5.show_weights(classifier, vec=vectorizer, top=50)

Weight?,Feature
+1.549,refreshing
+1.385,wonderfully
+1.313,erotic
+1.292,funniest
+1.288,excellent
+1.284,carrey
+1.283,perfect
+1.274,superb
+1.257,surprisingly
+1.240,appreciated


Посмотрим на конкретные примеры его работы:

In [20]:
print('Positive' if test_df['is_positive'].iloc[1] else 'Negative')
eli5.show_prediction(classifier, test_df['review'].iloc[1], vec=vectorizer, 
                     targets=['positive'], target_names=['negative', 'positive'])

Positive


In [21]:
print('Positive' if test_df['is_positive'].iloc[6] else 'Negative')
eli5.show_prediction(classifier, test_df['review'].iloc[6], vec=vectorizer, 
                     targets=['positive'], target_names=['negative', 'positive'])

Negative


Посмотрим на примеры неправильной классификации, наконец:

In [22]:
import numpy as np

preds = model.predict(test_df['review'])
incorrect_pred_index = np.random.choice(np.where(preds != test_df['is_positive'])[0])

eli5.show_prediction(classifier, test_df['review'].iloc[incorrect_pred_index],
                     vec=vectorizer, targets=['positive'], target_names=['negative', 'positive'])

## 1.  Проверьте повысилось ли качество на стандартных подходах при лемматизации/и без неё

Добавим лематизацию текста и посмотрим опвысится ли от этого качество. 

In [23]:
nlp = spacy.load("en_core_web_lg", disable=["ner"])

In [24]:
def lemmatize_text(text):
    doc = nlp(text)
    tokens=[token.lemma_.strip() for token in doc 
            #  if (not token.is_stop) or token.lemma_ in ['no','not']
            ]
    text=" ".join(tokens)
    return text

In [28]:
train_lem_df = train_df
test_lem_df = test_df

In [29]:
# Лемматизация текста. 
train_lem_df['review'] = train_lem_df['review'].progress_apply(lambda x: lemmatize_text(x))

100%|██████████| 25000/25000 [11:14<00:00, 37.07it/s]


In [30]:
test_lem_df['review'] = test_lem_df['review'].progress_apply(lambda x: lemmatize_text(x))

100%|██████████| 25000/25000 [16:15<00:00, 25.63it/s]


In [31]:
train_lem_df.head(5)

,is_positive,review
0,0,Dreamgirls despite its fistful of Tony win i...
1,0,this show come up with interesting location a...
2,1,I simply love this movie I also love the Ram...
3,0,spoiler ahead if you want to call they that ...
4,1,my all time favorite movie I have see many m...


In [32]:
with open('train_docs.pkl', 'wb') as f:
    pickle.dump(train_lem_df,f)
    
with open('test_docs.pkl', 'wb') as f: 
    pickle.dump(test_lem_df,f)

In [3]:
with open('train_docs.pkl', 'rb') as f:
    train_lem_df = pickle.load(f)
    
with open('test_docs.pkl', 'rb') as f:
    test_lem_df = pickle.load(f)

In [34]:
train_lem_df

,is_positive,review
0,0,Dreamgirls despite its fistful of Tony win i...
1,0,this show come up with interesting location a...
2,1,I simply love this movie I also love the Ram...
3,0,spoiler ahead if you want to call they that ...
4,1,my all time favorite movie I have see many m...
...,...,...
24995,1,I be a big fan of the movie but not for the ...
24996,0,I m not go to bother with a plot synopsis sin...
24997,0,this movie I do not know why they would tak...
24998,1,see this film on DVD yesterday and be gob sma...


In [35]:
vectorizer = CountVectorizer()
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(train_lem_df['review'], train_lem_df['is_positive'])

eval_model(model, test_lem_df)

Test accuracy = 85.97%


Test accuracy до лемматизизации равнялся = **86.69%**. Т.о. после лемматизации к-во даже немного ухудшилось. 

In [36]:
# Построим CountVectorizer на биграммах.
vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=20000, analyzer='word')
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(train_lem_df['review'], train_lem_df['is_positive'])

eval_model(model, test_lem_df)

Test accuracy = 88.28%


## 2.  Удалите/(замените на тег) из текстов сущности(имена, локации, что-то ещё). Запустите классификатор и модельки на сеточках

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [38]:
def tagging_text(text):
    doc = nlp(text)
    tokens=[token.ent_type_.strip() if token.ent_type_ !="" else token.text.strip() for token in doc ]

    # Удалим повотрояющиеся токены.
    text = [tokens[i] for i in range(1, len(tokens)) if tokens[i] != tokens[i-1] ]
    text=" ".join(text)
    return text

In [39]:
train_tag_df = train_lem_df
test_tag_df = test_lem_df

In [43]:
train_tag_df['review'] = train_tag_df['review'].progress_apply(lambda x: tagging_text(x))

100%|██████████| 25000/25000 [10:29<00:00, 39.73it/s]


In [44]:
test_tag_df['review'] = test_tag_df['review'].progress_apply(lambda x: tagging_text(x))

100%|██████████| 25000/25000 [16:16<00:00, 25.60it/s]


In [59]:
train_tag_df['review'][0]

'Dreamgirls  despite its fistful of PERSON win in DATE on FAC  have never be what one would call a jewel in the crown of stage musical  however  that be not to say that in the right cinematic hand it could not be flesh out and polish into something worthwhile on screen  unfortunately  what transfer to the screen be basically a slavishly faithful version of the stage hit with all of its inherent weakness intact  ORDINAL  the score have never be one of the strong point of this production and the film do not change that factor  there be lot of song  perhaps too many  but few of they be especially memorable  the close any come to catchy tune be the title song and one Night only  the much acclaimed and I be tell you that I be not go be less a great song than it be a dramatic set piece for the character of ORG  PERSON the film be slick and technically well produce  but the story and character be surprisingly thin and lack in any resonance  there be some interest in the opening moment  watch 

In [45]:
with open('train_tags.pkl', 'wb') as f:
    pickle.dump(train_tag_df,f)
    
with open('test_tags.pkl', 'wb') as f: 
    pickle.dump(test_tag_df,f)

In [8]:
with open('train_tags.pkl', 'rb') as f:
    train_tag_df = pickle.load(f)
    
with open('test_tags.pkl', 'rb') as f:
    test_tag_df = pickle.load(f)

In [9]:
vectorizer = CountVectorizer()
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(train_tag_df['review'], train_tag_df['is_positive'])

eval_model(model, test_tag_df)

Test accuracy = 86.66%


In [10]:
vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=20000, analyzer='word')
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(train_tag_df['review'], train_tag_df['is_positive'])

eval_model(model, test_tag_df)

Test accuracy = 88.56%


**Вывод:** После заммены текста на токены к-во немного подрасло. По сравнению с лемматизированным текстом. 

## Запустите классификатор и модельки на сеточках

Опробовал несколько вариантов. Оставил тот, которых дал лучший результат.

In [11]:
import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout, Conv1D, BatchNormalization, MaxPooling1D#, GlobalAveragePooling

## тегированный датасет.

In [12]:
from collections import Counter

words_counter = Counter((word for text in train_tag_df.review for word in text.lower().split()))

word2idx = {
    '': 0,
    '<unk>': 1
}
for word, count in words_counter.most_common():
    if count < 10:
        break
        
    word2idx[word] = len(word2idx)
    
print('Words count', len(word2idx))

Words count 13436


In [13]:
def convert(texts, word2idx, max_text_len):
    data = np.zeros((len(texts), max_text_len), dtype=np.int)
    
    for inx, text in enumerate(texts):
        result = []
        for word in text.split():
            if word in word2idx:
                result.append(word2idx[word])
        padding = [0]*(max_text_len - len(result))
        data[inx] = np.array(padding + result[-max_text_len:], dtype=np.int)
    return data

In [29]:
X_train = convert(train_tag_df.review, word2idx, 1000)
X_test = convert(test_tag_df.review, word2idx, 1000)

In [30]:
model = Sequential([
    Embedding(input_dim=len(word2idx), output_dim=256, input_shape=(X_train.shape[1],)),
    GlobalMaxPooling1D(),
    Dense(units=256, activation='relu'),
    Dropout(0.2),
    Dense(units=128, activation='relu'),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [31]:
model.fit(X_train, train_tag_df.is_positive, batch_size=1024, epochs=5, 
          validation_data=(X_test, test_tag_df.is_positive))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 50s 2ms/sample - loss: 0.6870 - acc: 0.5710 - val_loss: 0.6642 - val_acc: 0.5758
Epoch 2/5
25000/25000 [==============================] - 63s 3ms/sample - loss: 0.5410 - acc: 0.7990 - val_loss: 0.3584 - val_acc: 0.8529
Epoch 3/5
25000/25000 [==============================] - 65s 3ms/sample - loss: 0.2804 - acc: 0.8861 - val_loss: 0.2889 - val_acc: 0.8783
Epoch 4/5
25000/25000 [==============================] - 68s 3ms/sample - loss: 0.1835 - acc: 0.9297 - val_loss: 0.2844 - val_acc: 0.8838
Epoch 5/5
25000/25000 [==============================] - 55s 2ms/sample - loss: 0.1190 - acc: 0.9604 - val_loss: 0.3100 - val_acc: 0.8789


In [32]:
model.evaluate(X_test, test_tag_df.is_positive, batch_size=1024)

25000/25000 [==============================] - 8s 316us/sample - loss: 0.3100 - acc: 0.8789


[0.31001515754699704, 0.87892]

# Лемматизированный датасет. 

In [33]:
X_train = convert(train_lem_df.review, word2idx, 1000)
X_test = convert(test_lem_df.review, word2idx, 1000)

In [34]:
model = Sequential([
    Embedding(input_dim=len(word2idx), output_dim=256, input_shape=(X_train.shape[1],)),
    GlobalMaxPooling1D(),
    Dense(units=256, activation='relu'),
    Dropout(0.2),
    Dense(units=128, activation='relu'),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [35]:
model.fit(X_train, train_lem_df.is_positive, batch_size=1024, epochs=5, 
          validation_data=(X_test, test_lem_df.is_positive))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 61s 2ms/sample - loss: 0.6872 - acc: 0.5862 - val_loss: 0.6616 - val_acc: 0.8346
Epoch 2/5
25000/25000 [==============================] - 33s 1ms/sample - loss: 0.5254 - acc: 0.8396 - val_loss: 0.3373 - val_acc: 0.8620
Epoch 3/5
25000/25000 [==============================] - 34s 1ms/sample - loss: 0.2648 - acc: 0.8926 - val_loss: 0.2799 - val_acc: 0.8834
Epoch 4/5
25000/25000 [==============================] - 39s 2ms/sample - loss: 0.1748 - acc: 0.9352 - val_loss: 0.2896 - val_acc: 0.8815
Epoch 5/5
23552/25000 [===========================>..] - ETA: 1s - loss: 0.1111 - acc: 0.9646

In [ ]:
model.evaluate(X_test, test_lem_df.is_positive, batch_size=1024)

**Вывод:** Модель обученная на токенезированном датасете дала результат лучше чем когда используется только лемматизация.